In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Irritable_bowel_syndrome_(IBS)"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os
import pandas as pd

# 1. Find the most relevant directory for Irritable Bowel Syndrome (IBS)
subdirectories = os.listdir(tcga_root_dir)
target_trait = trait.lower().replace("_", " ")  # Convert to lowercase for case-insensitive matching

# Search for related terms to Irritable Bowel Syndrome
related_terms = ["bowel", "intestine", "colon", "gastro", "ibs", "colorectal"]
matched_dir = None

for subdir in subdirectories:
    subdir_lower = subdir.lower()
    # Check if any related term is in the directory name
    if any(term in subdir_lower for term in related_terms):
        print(f"Found potential match: {subdir}")
        matched_dir = subdir
        # If exact match found, select it
        if "bowel" in subdir_lower or "ibs" in subdir_lower:
            print(f"Selected as best match: {subdir}")
            matched_dir = subdir
            break

# If we found a potential match, use it
if matched_dir:
    print(f"Selected directory: {matched_dir}")
    
    # 2. Get the clinical and genetic data file paths
    cohort_dir = os.path.join(tcga_root_dir, matched_dir)
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic file: {os.path.basename(genetic_file_path)}")
    
    # 3. Load the data files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # 4. Print clinical data columns for inspection
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Print basic information about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")
    
    # Check if we have both gene and trait data
    is_gene_available = genetic_df.shape[0] > 0
    is_trait_available = clinical_df.shape[0] > 0
    
else:
    print(f"No suitable directory found for {trait}.")
    is_gene_available = False
    is_trait_available = False

# Record the data availability
validate_and_save_cohort_info(
    is_final=False,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Exit if no suitable directory was found
if not matched_dir:
    print("Skipping this trait as no suitable data was found.")


Found potential match: TCGA_Colon_and_Rectal_Cancer_(COADREAD)
Found potential match: TCGA_Colon_Cancer_(COAD)
Selected directory: TCGA_Colon_Cancer_(COAD)
Clinical file: TCGA.COAD.sampleMap_COAD_clinicalMatrix
Genetic file: TCGA.COAD.sampleMap_HiSeqV2_PANCAN.gz



Clinical data columns:
['AWG_MLH1_silencing', 'AWG_cancer_type_Oct62011', 'CDE_ID_3226963', 'CIMP', 'MSI_updated_Oct62011', '_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mirna_COAD', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'age_at_initial_pathologic_diagnosis', 'anatomic_neoplasm_subdivision', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'braf_gene_analysis_performed', 'braf_gene_analysis_result', 'circumferential_resection_margin', 'colon_polyps_present', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'disease_code', 'foll

### Step 2: Find Candidate Demographic Features

In [3]:
# 1. Identify candidate age and gender columns
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
candidate_gender_cols = ['gender']

# 2. Load clinical data to preview candidate columns
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(os.path.join(tcga_root_dir, 'TCGA_Colon_Cancer_(COAD)'))
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Preview age columns
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].head(5).tolist()

print("Age columns preview:")
print(age_preview)

# Preview gender columns
gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].head(5).tolist()

print("\nGender columns preview:")
print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [61.0, 67.0, 42.0, 74.0, nan], 'days_to_birth': [-22379.0, -24523.0, -15494.0, -27095.0, nan]}

Gender columns preview:
{'gender': ['FEMALE', 'MALE', 'FEMALE', 'MALE', nan]}


### Step 3: Select Demographic Features

In [4]:
# Examine the age columns and select the most appropriate one
age_col = None
gender_col = None

# Check age columns
age_columns_data = {'age_at_initial_pathologic_diagnosis': [61.0, 67.0, 42.0, 74.0, float('nan')], 
                    'days_to_birth': [-22379.0, -24523.0, -15494.0, -27095.0, float('nan')]}

# Both age columns have similar information but 'age_at_initial_pathologic_diagnosis' is more directly interpretable
# as it's already in years, while 'days_to_birth' is negative days and would need conversion
age_col = 'age_at_initial_pathologic_diagnosis'

# Check gender column
gender_columns_data = {'gender': ['FEMALE', 'MALE', 'FEMALE', 'MALE', float('nan')]}

# Only one gender column is available and it has valid values
gender_col = 'gender'

# Print the chosen columns
print(f"Chosen age column: {age_col}")
print(f"Chosen gender column: {gender_col}")


Chosen age column: age_at_initial_pathologic_diagnosis
Chosen gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Create clinical features dataframe with trait (Canavan Disease) using patient IDs
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# Step 2: Normalize gene symbols in the gene expression data
# The gene symbols in TCGA genetic data are already standardized, but we'll normalize them for consistency
normalized_gene_df = normalize_gene_symbols_in_index(genetic_df)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")
print(f"Gene expression data shape after normalization: {normalized_gene_df.shape}")

# Step 3: Link clinical and genetic data
# Transpose genetic data to have samples as rows and genes as columns
genetic_df_t = normalized_gene_df.T
# Save the clinical data for reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_features.shape}")

# Verify common indices between clinical and genetic data
clinical_indices = set(clinical_features.index)
genetic_indices = set(genetic_df_t.index)
common_indices = clinical_indices.intersection(genetic_indices)
print(f"Number of samples in clinical data: {len(clinical_indices)}")
print(f"Number of samples in genetic data: {len(genetic_indices)}")
print(f"Number of common samples: {len(common_indices)}")

# Link the data by using the common indices
linked_data = pd.concat([clinical_features.loc[list(common_indices)], genetic_df_t.loc[list(common_indices)]], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# Step 4: Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# Step 5: Determine whether the trait and demographic features are severely biased
trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait=trait)

# Step 6: Conduct final quality validation and save information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=linked_data,
    note=f"Dataset contains TCGA glioma and brain tumor samples with gene expression and clinical information for {trait}."
)

# Step 7: Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable based on validation criteria. Data not saved.")

print("Preprocessing completed.")

Normalized gene expression data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/gene_data/TCGA.csv
Gene expression data shape after normalization: (19848, 329)
Clinical data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/clinical_data/TCGA.csv
Clinical data shape: (551, 3)
Number of samples in clinical data: 551
Number of samples in genetic data: 329
Number of common samples: 329
Linked data shape: (329, 19851)


Data shape after handling missing values: (329, 19851)
For the feature 'Irritable_bowel_syndrome_(IBS)', the least common label is '0' with 41 occurrences. This represents 12.46% of the dataset.
The distribution of the feature 'Irritable_bowel_syndrome_(IBS)' in this dataset is fine.

Quartiles for 'Age':
  25%: 56.0
  50% (Median): 68.0
  75%: 77.0
Min: 31.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 150 occurrences. This represents 45.59% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/TCGA.csv
Preprocessing completed.
